# 🚀 dLNk GPT - Production Training System (ฉบับสมบูรณ์)

## ระบบเทรนอัตโนมัติแบบครบวงจร

### ✨ ฟีเจอร์ครบครัน:
- ✅ ดึงข้อมูล Exploit/Payload จากแหล่งจริง (Exploit-DB, PayloadsAllTheThings, SecLists)
- ✅ ป้องกัน Overfitting อัตโนมัติ
- ✅ ตรวจจับ Model ไม่เรียนรู้
- ✅ ตรวจสอบข้อมูลขัดแย้ง
- ✅ ปรับ Learning Rate อัตโนมัติ
- ✅ แจ้งเตือนภาษาไทยผ่าน Telegram แบบเรียลไทม์
- ✅ บันทึกผลลัพธ์ไปยัง Weights & Biases
- ✅ รัน Test Mode ก่อน Production

### 📋 ขั้นตอนการใช้งาน:
1. เปลี่ยน Runtime เป็น GPU (T4 หรือ A100)
2. รัน cells ทั้งหมดตามลำดับ
3. ระบบจะรัน Test Mode ก่อนอัตโนมัติ
4. หลังจาก Test ผ่าน จะเริ่มเทรนจริง
5. รับการแจ้งเตือนผ่าน Telegram

---

**⚠️ หมายเหตุสำคัญ:**
- การเทรนจะใช้เวลา 2-4 ชั่วโมง (ขึ้นอยู่กับ GPU)
- ต้องเปิด Colab tab ไว้ตลอด (ปิด browser ได้ แต่ต้องเปิด tab)
- แนะนำให้ใช้ Colab Pro สำหรับ GPU ที่ดีกว่า

## 1️⃣ ตรวจสอบ GPU

In [ ]:
!nvidia-smi
print("\n✅ GPU พร้อมใช้งาน")

## 2️⃣ ติดตั้ง Packages

In [ ]:
%%capture
!pip install -q transformers>=4.30.0 datasets>=2.12.0 accelerate>=0.20.0 peft>=0.4.0 bitsandbytes tensorboard wandb requests
print("✅ ติดตั้ง packages เสร็จแล้ว")

## 4️⃣ Login Hugging Face

In [ ]:
from huggingface_hub import login

HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # 👈 ใส่ token ของคุณที่นี่

if not HF_TOKEN or HF_TOKEN == "YOUR_HF_TOKEN":
    print("⚠️  กรุณาใส่ Hugging Face token")
else:
    login(token=HF_TOKEN)
    print("✅ Login Hugging Face สำเร็จ")

## 5️⃣ Login Weights & Biases (Optional)

In [ ]:
import wandb

WANDB_API_KEY = ""  # 👈 ใส่ W&B API key ที่นี่ (หรือเว้นว่างไว้)

if WANDB_API_KEY:
    wandb.login(key=WANDB_API_KEY)
    print("✅ Login W&B สำเร็จ")
else:
    print("⚠️  ข้าม W&B login")

## 6️⃣ Clone Repository

In [ ]:
!git clone https://github.com/traingptproject/gptprojecttrain.git
%cd gptprojecttrain
!ls -la
print("\n✅ Clone repository สำเร็จ")

## 7️⃣ ตั้งค่า Telegram

### 🔍 วิธีหา Chat ID:
1. เปิด Telegram และค้นหา Bot: `@apidlnkbot`
2. กด Start หรือส่งข้อความอะไรก็ได้
3. เปิด browser ไปที่: https://api.telegram.org/bot8505747217:AAHpbWUfgrcQgJcK1JnqK9sds9nSwsTqVBg/getUpdates
4. หา `"chat":{"id": XXXXXXX}` ในผลลัพธ์
5. นำ Chat ID มาใส่ด้านล่าง

In [ ]:
import os
import requests

TELEGRAM_BOT_TOKEN = "8505747217:AAHpbWUfgrcQgJcK1JnqK9sds9nSwsTqVBg"
TELEGRAM_CHAT_ID = "YOUR_REAL_CHAT_ID"  # 👈 ใส่ Chat ID ที่ได้จาก getUpdates

def send_telegram_message(message):
    """ส่งข้อความไปยัง Telegram"""
    if TELEGRAM_CHAT_ID == "YOUR_REAL_CHAT_ID":
        print("⚠️  ไม่ได้ตั้งค่า Telegram Chat ID")
        return False
    
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
        response = requests.post(url, data=data, timeout=10)
        return response.status_code == 200
    except Exception as e:
        print(f"❌ ส่ง Telegram ไม่สำเร็จ: {e}")
        return False

# ทดสอบส่งข้อความ
if TELEGRAM_CHAT_ID != "YOUR_REAL_CHAT_ID":
    os.environ['TELEGRAM_BOT_TOKEN'] = TELEGRAM_BOT_TOKEN
    os.environ['TELEGRAM_CHAT_ID'] = TELEGRAM_CHAT_ID
    
    if send_telegram_message("🚀 <b>dLNk GPT Training System</b>\n\n✅ เชื่อมต่อ Telegram สำเร็จ\nระบบพร้อมเริ่มการเทรน"):
        print("✅ ทดสอบส่ง Telegram สำเร็จ")
    else:
        print("❌ ส่ง Telegram ไม่สำเร็จ - ตรวจสอบ Chat ID")
else:
    print("❌ กรุณาใส่ Telegram Chat ID ที่ถูกต้อง!")
    print("   ดูวิธีหา Chat ID ในคำอธิบายด้านบน")

## 8️⃣ ดาวน์โหลดข้อมูล Exploit และ Payload

ระบบจะดาวน์โหลดข้อมูลจาก:
- Exploit-DB
- PayloadsAllTheThings
- SecLists (บางส่วน)

In [ ]:
import os
import subprocess

print("📥 กำลังดาวน์โหลดข้อมูล Exploit และ Payload...\n")

# สร้างโฟลเดอร์สำหรับเก็บข้อมูล
os.makedirs("/content/exploit_data", exist_ok=True)
os.chdir("/content/exploit_data")

# 1. Clone Exploit-DB (เฉพาะ exploits และ shellcodes)
print("1️⃣ ดาวน์โหลด Exploit-DB...")
!git clone --depth 1 --filter=blob:none --sparse https://gitlab.com/exploit-database/exploitdb.git
%cd exploitdb
!git sparse-checkout set exploits shellcodes files_exploits.csv files_shellcodes.csv
print("✅ ดาวน์โหลด Exploit-DB เสร็จแล้ว\n")

# 2. Clone PayloadsAllTheThings
%cd /content/exploit_data
print("2️⃣ ดาวน์โหลด PayloadsAllTheThings...")
!git clone --depth 1 https://github.com/swisskyrepo/PayloadsAllTheThings.git
print("✅ ดาวน์โหลด PayloadsAllTheThings เสร็จแล้ว\n")

# 3. Clone SecLists (เฉพาะ Fuzzing payloads)
print("3️⃣ ดาวน์โหลด SecLists (Fuzzing)...")
!git clone --depth 1 --filter=blob:none --sparse https://github.com/danielmiessler/SecLists.git
%cd SecLists
!git sparse-checkout set Fuzzing
print("✅ ดาวน์โหลด SecLists เสร็จแล้ว\n")

%cd /content/gptprojecttrain
print("\n✅ ดาวน์โหลดข้อมูลทั้งหมดเสร็จสิ้น")
print(f"📊 ขนาดข้อมูลทั้งหมด: {subprocess.check_output(['du', '-sh', '/content/exploit_data']).decode().split()[0]}")

## 9️⃣ ประมวลผลและรวมข้อมูล

แปลงข้อมูล Exploit/Payload เป็นรูปแบบ Training Dataset

In [ ]:
import json
import os
import csv
from pathlib import Path

print("🔄 กำลังประมวลผลข้อมูล...\n")

training_data = []

# 1. ประมวลผล Exploit-DB
print("1️⃣ ประมวลผล Exploit-DB...")
exploitdb_csv = "/content/exploit_data/exploitdb/files_exploits.csv"
if os.path.exists(exploitdb_csv):
    with open(exploitdb_csv, 'r', encoding='utf-8', errors='ignore') as f:
        reader = csv.DictReader(f)
        for row in list(reader)[:1000]:  # เอาแค่ 1000 รายการแรก
            if row.get('file') and row.get('description'):
                file_path = f"/content/exploit_data/exploitdb/{row['file']}"
                if os.path.exists(file_path):
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as exploit_file:
                            code = exploit_file.read()[:5000]  # เอาแค่ 5000 ตัวอักษรแรก
                            training_data.append({
                                "instruction": f"อธิบาย exploit: {row['description']}",
                                "input": f"Platform: {row.get('platform', 'Unknown')}, Type: {row.get('type', 'Unknown')}",
                                "output": f"Exploit Code:\n{code}"
                            })
                    except:
                        pass
    print(f"   ✅ ประมวลผล {len(training_data)} exploits จาก Exploit-DB")

# 2. ประมวลผล PayloadsAllTheThings
print("\n2️⃣ ประมวลผล PayloadsAllTheThings...")
payloads_dir = "/content/exploit_data/PayloadsAllTheThings"
count = 0
for md_file in Path(payloads_dir).rglob("*.md"):
    if count >= 500:  # จำกัดที่ 500 ไฟล์
        break
    try:
        with open(md_file, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()[:3000]
            if len(content) > 100:
                training_data.append({
                    "instruction": f"อธิบายเทคนิค: {md_file.stem}",
                    "input": f"หมวดหมู่: {md_file.parent.name}",
                    "output": content
                })
                count += 1
    except:
        pass
print(f"   ✅ ประมวลผล {count} payloads จาก PayloadsAllTheThings")

# 3. บันทึกเป็น JSONL
output_file = "/content/drive/MyDrive/Colab Notebooks/training_data_combined.jsonl"
print(f"\n💾 บันทึกข้อมูลไปยัง: {output_file}")
with open(output_file, 'w', encoding='utf-8') as f:
    for item in training_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"\n✅ ประมวลผลเสร็จสิ้น")
print(f"📊 จำนวนข้อมูลทั้งหมด: {len(training_data):,} samples")
print(f"📁 ไฟล์: {output_file}")

## 🔟 รัน Test Mode (ทดสอบก่อนเทรนจริง)

ระบบจะทดสอบด้วยข้อมูล 100 samples เพื่อตรวจสอบว่าทุกอย่างทำงานถูกต้อง

In [ ]:
print("🧪 เริ่มต้น Test Mode...\n")

# ส่งการแจ้งเตือนไปยัง Telegram
send_telegram_message(
    "🧪 <b>Test Mode เริ่มต้น</b>\n\n"
    "กำลังทดสอบระบบด้วยข้อมูล 100 samples\n"
    "ใช้เวลาประมาณ 2-3 นาที"
)

# รัน test script
!python train_test_monitored.py

print("\n✅ Test Mode เสร็จสิ้น")
send_telegram_message(
    "✅ <b>Test Mode สำเร็จ</b>\n\n"
    "ระบบทำงานปกติ\n"
    "พร้อมเริ่มการเทรนจริง"
)

## 1️⃣1️⃣ เริ่มการเทรนจริง (Production Mode)

**⚠️ สำคัญ:**
- การเทรนจะใช้เวลา 2-4 ชั่วโมง
- ระบบจะแจ้งเตือนผ่าน Telegram ทุก epoch
- มีระบบป้องกัน Overfitting อัตโนมัติ
- สามารถหยุดการเทรนได้ตลอดเวลา

In [ ]:
print("🚀 เริ่มการเทรนจริง (Production Mode)...\n")
print("⏰ เวลาโดยประมาณ: 2-4 ชั่วโมง")
print("📱 คุณจะได้รับการแจ้งเตือนผ่าน Telegram\n")

# ส่งการแจ้งเตือนไปยัง Telegram
send_telegram_message(
    "🚀 <b>Production Training เริ่มต้น</b>\n\n"
    "⏰ เวลาโดยประมาณ: 2-4 ชั่วโมง\n"
    "📊 คุณจะได้รับอัปเดตทุก epoch\n\n"
    "ระบบมีการป้องกัน:\n"
    "✅ Overfitting\n"
    "✅ Model ไม่เรียนรู้\n"
    "✅ ข้อมูลขัดแย้ง\n"
    "✅ Auto Learning Rate Adjustment"
)

# รัน production training script
!python train_monitored_v2.py

print("\n🎉 การเทรนเสร็จสิ้น!")
send_telegram_message(
    "🎉 <b>การเทรนเสร็จสิ้น!</b>\n\n"
    "✅ โมเดลถูกบันทึกแล้ว\n"
    "✅ ผลลัพธ์อัปโหลดไปยัง Hugging Face\n"
    "✅ สามารถตรวจสอบผลลัพธ์ใน W&B ได้\n\n"
    "ขอบคุณที่ใช้ dLNk GPT Training System!"
)

## 🎉 เสร็จสิ้น!

### สิ่งที่คุณได้รับ:
1. **โมเดลที่เทรนแล้ว** - อัปโหลดไปยัง Hugging Face Hub
2. **ผลลัพธ์การเทรน** - บันทึกใน Weights & Biases
3. **การแจ้งเตือน** - ผ่าน Telegram ตลอดการเทรน
4. **รายงานสรุป** - ระบบป้องกันปัญหาต่างๆ

### ขั้นตอนต่อไป:
- ทดสอบโมเดลที่เทรนแล้ว
- วิเคราะห์ผลลัพธ์ใน W&B
- ปรับแต่ง hyperparameters ถ้าจำเป็น
- เทรนต่อด้วยข้อมูลเพิ่มเติม

---

**สร้างโดย:** dLNk GPT Training System  
**เวอร์ชัน:** Production v1.0  
**วันที่:** 2025